<a href="https://colab.research.google.com/github/sindhujajakkula/CFD_Startup_laminar_flow_projection-_method/blob/main/CFD_Startup_laminar_flow_projection_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as pyplot
import numpy as np
from itertools import repeat

R_POINTS = 50
L_POINTS = 100
PIPE_RADIUS = 0.5
PIPE_LENGTH = 1.0
TIME_STEP_LENGTH = 0.001
KINEMATIC_VISCOSITY = 0.1
DENSITY = 1.0
U_INITIAL = 1.0
N_PRESSURE_POISSON_ITERATIONS = 50

def main():
  
  element_length_z = PIPE_LENGTH/(L_POINTS-1)
  element_length_r = 2*PIPE_RADIUS/(R_POINTS-1)

  r = np.linspace(0, 2*PIPE_RADIUS, R_POINTS)
  z = np.linspace(0.0, PIPE_LENGTH, L_POINTS)

  Z, R = np.meshgrid(z, r)

  u_prev = np.zeros_like(Z)
  p_prev = np.zeros_like(Z)

  r=[]
  r_value = np.linspace(PIPE_RADIUS, -PIPE_RADIUS, R_POINTS)
  for i in range(len(r_value)):
    nl=[]
    nl.extend(repeat(r_value[i],L_POINTS))
    r.append(nl)
  r = np.array(r)

  #defining functions
  def central_difference_z(f):
    #calculating ∂f/∂z
    diff = np.zeros_like(f)
    diff[1:-1, 1:-1] = ( f[1:-1, 2:] - f[1:-1, 0:-2] )/(2*element_length_z)
    return diff 
  

  def central_difference_r(f):
    #calculating 1/r ∂f/∂r
    diff = np.zeros_like(f)
    diff_num = f[2:,  1:-1] - f[0:-2, 1:-1]
    diff_denom = r[1:-1,  1:-1]
    diff[1:-1, 1:-1] = ( np.divide(diff_num, diff_denom) )/(2*element_length_r)
    return diff

  def laplace(f):
    diff = np.zeros_like(Z)
    diff[1:-1, 1:-1] = ( 
        central_difference_r(f)[1:-1, 1:-1]    
        +
        ( f[1:-1, 2:] + f[1:-1, 0:-2] - 2*f[1:-1, 1:-1] )/( element_length_r**2 )
        +
        ( f[2: , 1:-1] + f[0:-2, 1:-1] - 2*f[1:-1, 1:-1] )/( element_length_z**2 )
    ) 
    return diff



  A = True 
  while A:
    d_u_prev__d_z = central_difference_z(u_prev)
    laplace__u_prev = laplace(u_prev)
    laplace__p_prev = laplace(p_prev)
    print(laplace__p_prev)

    u_tent = (
        u_prev
        +
        TIME_STEP_LENGTH*(
            -u_prev*d_u_prev__d_z
            +
            KINEMATIC_VISCOSITY*laplace__u_prev
        )
    )

    #Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
    u_tent[:, 0] = U_INITIAL #left boundary, for all points along r at z= 0
    u_tent[-1, : ] = 0.0 #top boundary, for all points along z at r=R
    u_tent[0, :] = 0.0 #bottom boundary, for all points along z at r=R
    u_tent[:, -1] = 0.0 #right boundary,for all points along r at z= L

    d_u_tent__d_z = central_difference_z(u_tent)

    # Compute a pressure correction by solving the pressure-poisson equation
    rhs = (
        (DENSITY / TIME_STEP_LENGTH)*d_u_tent__d_z
    )
    
    for _ in range(N_PRESSURE_POISSON_ITERATIONS):
      p_next = np.zeros_like(p_prev)

      c = 1/(2*r*(element_length_z**2 + element_length_r**2))

      p_next[1:-1, 1:-1] = c*(
          element_length_z**2*np.multiply(( r[1:-1  , 1:-1] + element_length_r), p_prev[2:  , 1:-1])
          +
          element_length_z**2*np.multiply((r[1:-1 , 1:-1] - element_length_r),p_prev[0:-2, 1:-1])
          +
          r[1:-1 , 1:-1]*(element_length_r**2)*(p_prev[1:-1, 2: ])
          +
          r[1:-1 , 1:-1]*(element_length_r**2)*(p_prev[1:-1, 0:-2])
          +
          r[1:-1 , 1:-1]*(element_length_r**2)*(element_length_z**2)*rhs[1:-1, 1:-1]
      )

      # Pressure Boundary Conditions: Homogeneous Neumann Boundary conditions
      p_next[:, -1] = 0.0 #pressure at outlet is zero
      p_next[0,  :] = p_next[1,  :]
      p_next[:,  0] = p_next[:,  1]
      p_next[-1, :] = p_next[-2, :]


      p_prev = p_next

    d_p_next__d_z = central_difference_z(p_next)

      # Correct the velocities such that the fluid stays incompressible
    u_next = (
        
          u_tent
            -
            TIME_STEP_LENGTH / DENSITY
            *
            d_p_next__d_z
    )      
    # Velocity Boundary Conditions: Homogeneous Dirichlet BC everywhere
    # except for the horizontal velocity at the top, which is prescribed
    u_next[0, :] = 0.0
    u_next[:, 0] = U_INITIAL
    u_next[:, -1] = 0.0#right boundary
    u_next[-1, :] = 0.0

    if u_next - u_prev == 0:
      #iteration continues till steady state is achieved
      A = False

    # Advance in time
    u_prev = u_next
    p_prev = p_next

    # The [::2, ::2] selects only every second entry (less cluttering plot
  plt.style.use("dark_background")
  plt.figure()
  plt.contourf(Z[::2, ::2], R[::2, ::2], p_next[::2, ::2], cmap="coolwarm")
  plt.colorbar()

  
  plt.streamplot(X[::2, ::2], Y[::2, ::2], u_next[::2, ::2], v_next[::2, ::2], color="black")
  plt.xlim((0, 1))
  plt.ylim((0, 1))
  plt.show()


if __name__ == "__main__":
    main()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


ValueError: ignored